In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, '/Users/joe/pycrash/')

In [3]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.io as pio
import importlib
%matplotlib inline

In [4]:
from IPython.display import clear_output
import time 
import pandas as pd
import numpy as np
from PIL import Image
import pickle
import sys

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
from pycrash.impact_main import Impact
from pycrash.model_calcs.carpenter_momentum import IMPC
from pycrash.visualization.initial_positions import initial_position
from pycrash.model_calcs.position_data import position_data_static
from pycrash.vehicle import Vehicle
import pycrash.functions.ar as ar

Current values for defined constants:
maximum available friction (mu_max) = 0.8
time step for vehicle motion (dt) = 0.001 s


In [6]:
# load background image
image_dict = {'image': Image.open('/Users/joe/google_site_1920_1080.png'),
              'xOffset': -170,
              'yOffset': -120,
              'pixelsPerFoot': 3.48,
              'opacity': 0.8}

In [7]:
# vehicle database
with open('/Users/joe/vehicledata.pkl', 'rb') as handle:
    vehicledata = pickle.load(handle)

### Create Vehicles

In [114]:
vehicle_inputs = vehicledata['2017 Gillig Bus']
veh3 = Vehicle('Bus', input_dict=vehicle_inputs)
veh3.hcg = 3
veh3.lcgf = veh3.wb / 2
veh3.lcgr = veh3.wb / 2
veh3.track = 7.5
veh3.tire_d = 3
veh3.tire_w = 10/12
veh3.f_hang = 7.5
veh3.r_hang = 7.5
veh3.izz = 750000 / 3
veh3.striking = True
veh3.isTrailer = False

Input entry modelYearStart unknown, setting to 1997
Input entry modelYearEnd unknown, setting to 2021
Input entry GVWR unknown, setting to 33000
Vehicle inputs for Bus applied successfully


### Create Tractor Trailer

In [115]:
# tractor
newVehicleName = '1999 Freightliner Long Conv'
vehicle_inputs = {"year": 1999,
              "make": "Freightliner",
              "model": "Long Conv",
              "weight": 52000,
              "gvwr": 52000,
              "steer_ratio": 20,
              "init_x_pos": 0,
              "init_y_pos": 0,
              "head_angle": 0,
              "width": 2.6 * 3.28084,
              "length": 8 * 3.28084,
              "hcg": 4,
              "lcgf": 3 * 3.28084,
              "lcgr": 3 * 3.28084,
              "wb": 6 * 3.28084,
              "track": 2.1 * 3.28084,
              "f_hang": 0.8 * 3.28084,
              "r_hang": 1.3 * 3.28084,
              "tire_d": 1 * 3.28084,
              "tire_w": 0.3 * 3.28084,
              "izz": 10000,
              "fwd": 0,
              "rwd": 1,
              "awd": 0,
              "vx_initial": 0,
              "vy_initial": 0,
              "omega_z": 0}

veh1 = Vehicle('Tractor', input_dict=vehicle_inputs)
veh1.striking = False
veh1.isTrailer = False
veh1.cg_kp = 3.22 * 3.28084

Input entry gvwr unknown, setting to 52000
Vehicle inputs for Tractor applied successfully


In [116]:
# trailer
newVehicleName = 'forty eight trailer'
vehicle_inputs = {"year": 1999,
                  "make": "trailer",
                  "model": "forty eight",
                  "weight": 30000,
                  "gvwr": 30000,
                  "init_x_pos": 0,
                  "init_y_pos": 0,
                  "head_angle": 0,
                  "width": 2.7 * 3.28084,
                  "length": 16 * 3.28084,
                  "kp_axle": 12 * 3.28084,    # <- kingpin to axle
                  "track": 2.4 * 3.28084,
                  "f_hang": 1.1 * 3.28084,
                  "r_hang": 3.2 * 3.28084,
                  "tire_d": 1 * 3.28084,
                  "tire_w": 0.3 * 3.28084,
                  "izz": 750000,
                  "vx_initial": 0,
                  "vy_initial": 0,
                  "omega_z": 0}

veh2 = Vehicle('Trailer', input_dict=vehicle_inputs)
veh2.lcgf = (veh2.kp_axle / 2)
veh2.lcgr = (veh2.kp_axle / 2)
veh2.isTrailer = True
veh2.striking = True

Input entry gvwr unknown, setting to 30000
Input entry kp_axle unknown, setting to 39.37008
Vehicle inputs for Trailer applied successfully


### Vehicle Inputs

Tractor

In [117]:
veh1.vx_initial = 15

Bus

In [118]:
veh3.vx_initial = 0

### Vehicle impact points and impact planes manually entered
- `impact_points` is a list of impacts points in the order that they will occurr in the simulation
- `edgeimpact_points` is a list of points defining the impact edge clockwise
- each impact requires a defined point and plane even if there is no change

In [119]:
# vehicle 2 (Trailer)
veh2.impactTotal = 1  # <- number of impacts for this vehicle
veh2.edgeimpact = [3] * veh2.impactTotal
# 
veh2.edgeimpact_points = [(-1 * veh2.lcgr - veh2.r_hang, -1 * veh2.width / 2, veh2.lcgf + veh2.f_hang, -veh2.width / 2)] * veh2.impactTotal
veh2.head_angle = 250
veh2.striking = False


# vehicle 3
veh3.impactTotal = 1  # <- number of impacts for this vehicle
# impact point = (x, y, impact plane normal angle [deg])
veh3.impact_points = [(-veh3.lcgr - veh3.r_hang, -veh3.width / 2, 70)]
veh3.head_angle = 90
veh3.striking = True

### Calculate trailer conditions and impact

Kingpin velocity in global frame

In [120]:
# kingpin velocity in tractor frame
veh1.cg_kp = 3.22 * 3.28084
veh1.head_angle = 195
kp_Vx = veh1.vx_initial * 1.46667 * np.cos(veh1.head_angle * np.pi / 180) - veh1.vy_initial * 1.46667 * np.sin(veh1.head_angle * np.pi / 180)
kp_Vy = veh1.vx_initial * 1.46667 * np.sin(veh1.head_angle * np.pi / 180) + veh1.vy_initial * 1.46667 * np.cos(veh1.head_angle * np.pi / 180)
print(f'Initial Kingpin velocity -> x: {kp_Vx * 0.681818:0.2f}, y: {kp_Vy * 0.681818:0.2f}')

Initial Kingpin velocity -> x: -14.49, y: -3.88


Trailer CG velocity

In [121]:
veh2.head_angle = 250
kp_vx = kp_Vx * np.cos(veh2.head_angle * np.pi / 180) - kp_Vy * np.sin(veh2.head_angle * np.pi / 180)
kp_vy = -1 * kp_Vx * np.sin(veh2.head_angle * np.pi / 180) + kp_Vy * np.cos(veh2.head_angle * np.pi / 180)
print(f'Initial Kingpin velocity in trailer frame -> x: {kp_vx * 0.681818:0.2f}, y: {kp_vy * 0.681818:0.2f}')

Initial Kingpin velocity in trailer frame -> x: 1.31, y: -12.29


Trailer CG velocity accounting for rotation

In [122]:
veh2.omega_z = kp_vy / veh2.kp_axle
print(f'Initial trailer omega: {veh2.omega_z :0.2f}')

# trailer cg velocity
veh2.vx_initial = kp_vx * 0.681818
veh2.vy_initial = (kp_vy - veh2.lcgf * veh2.omega_z) * 0.681818
print(f'Initial Trailer velocity -> x: {veh2.vx_initial:0.2f}, y: {veh2.vy_initial:0.2f} mph')

Initial trailer omega: -0.46
Initial Trailer velocity -> x: 1.31, y: -6.14 mph


### Get initial positions based on impact point

In [123]:
# Vehicle 2 initial position based on impact point
veh2_dx = 5
veh2_dy = -1 * veh2.width / 2

v2_hangle_rad = veh2.head_angle * np.pi / 180
veh2.init_x_pos = -1 * (veh2_dx * np.cos(v2_hangle_rad) - veh2_dy * np.sin(v2_hangle_rad))
veh2.init_y_pos = -1 * (veh2_dx * np.sin(v2_hangle_rad) + veh2_dy * np.cos(v2_hangle_rad))

# impact point in vehicle 2 reference frame
veh2.pimpact_x = veh2_dx
veh2.pimpact_y = veh2_dy

In [124]:
# Vehicle 1 based on location of trailer
kp_Dx = veh2.init_x_pos + veh2.lcgf * np.cos(veh2.head_angle * np.pi / 180)
kp_Dy = veh2.init_y_pos + veh2.lcgf * np.sin(veh2.head_angle * np.pi / 180)

veh1.init_x_pos = kp_Dx + veh1.cg_kp * np.cos(veh1.head_angle * np.pi / 180)
veh1.init_y_pos = kp_Dy + veh1.cg_kp * np.sin(veh1.head_angle * np.pi / 180)

In [125]:
# Vehicle 3 initial position based on impact point
veh3_dx = -1 * veh3.lcgr - veh3.r_hang
veh3_dy = -1 * veh3.width / 2

v3_hangle_rad = veh3.head_angle * np.pi / 180
veh3.init_x_pos = -1 * (veh3_dx * np.cos(v3_hangle_rad) - veh3_dy * np.sin(v3_hangle_rad))
veh3.init_y_pos = -1 * (veh3_dx * np.sin(v3_hangle_rad) + veh3_dy * np.cos(v3_hangle_rad))

# impact point in vehicle 3 reference frame
veh3.pimpact_x = veh3_dx
veh3.pimpact_y = veh3_dy

### Initial Positions

In [126]:
initial_position(position_data_static([veh1, veh2, veh3]))

### Create Simulation

In [127]:
# inputs for simulation
sim_inputs = {'cor': 0.2,
             'cof': 0.3,
             'impact_norm_deg': 70}

In [128]:
name = 'Bus vs Trailer'
run = IMPC(name, veh2, veh3, sim_inputs)

theta c: 410.00 [deg]
p-ratio: 0.42, cof: 0.3
Sliding Condition
Compression
Reverse Slide
Impact Energy: {'t_effects_dis': 2312.486479620179, 'n_effects_dis': 9877.709736541163, 'tn_total_dis': 12190.196216161341}


In [129]:
run.__dict__

{'name': 'Bus vs Trailer',
 'type': 'impc_carpenter',
 'veh1': <pycrash.vehicle.Vehicle at 0x176f02f90>,
 'veh2': <pycrash.vehicle.Vehicle at 0x176f31350>,
 'poi_veh2x': -17.291666666666664,
 'poi_veh2y': -4.25,
 'v1_result': {'vx': 1.4455317545330766,
  'vy': -3.7475580405494306,
  'oz_rad': 0.014953916710315589,
  'oz_deg': 0.8567963146912394,
  'dvx': 0.13819036812344693,
  'dvy': 2.3961069406649806,
  'dv': 2.4000885502299676},
 'v2_result': {'vx': 1.11516050328014,
  'vy': 2.5892797038899436,
  'oz_rad': -0.1862860911129808,
  'oz_deg': -10.673406802763312,
  'dvx': 1.11516050328014,
  'dvy': 2.5892797038899436,
  'dv': 2.819211296276391}}

### Parametric Analysis
- tractor initial speed 9 - 11
- trailer heading angle 240 - 260
- tractor heading angle 185 - 205
- impact plane angle

In [142]:
monte_carlo_impacts = pd.DataFrame()

for n_run in range(1, 1000):
    # tractor velocity
    veh1.vx_initial = np.random.uniform(9,11)
    
    # vehicle orientations
    # vehicle 2 (Trailer)
    veh2.impactTotal = 1  # <- number of impacts for this vehicle
    veh2.edgeimpact = [3] * veh2.impactTotal
    # 
    veh2.edgeimpact_points = [(-1 * veh2.lcgr - veh2.r_hang, -1 * veh2.width / 2, veh2.lcgf + veh2.f_hang, -veh2.width / 2)] * veh2.impactTotal
    veh2.head_angle = np.random.uniform(240, 260)
    veh2.striking = False
    
    impact_norm_angle = np.random.uniform(60, 80)
    
    # vehicle 3
    veh3.impactTotal = 1  # <- number of impacts for this vehicle
    # impact point = (x, y, impact plane normal angle [deg])
    veh3.impact_points = [(-veh3.lcgr - veh3.r_hang, -veh3.width / 2, 70)]
    veh3.head_angle = 90
    veh3.striking = True
    
    # kingpin velocity in tractor frame
    veh1.cg_kp = 3.22 * 3.28084
    veh1.head_angle = np.random.uniform(185, 205)
    kp_Vx = veh1.vx_initial * 1.46667 * np.cos(veh1.head_angle * np.pi / 180) - veh1.vy_initial * 1.46667 * np.sin(veh1.head_angle * np.pi / 180)
    kp_Vy = veh1.vx_initial * 1.46667 * np.sin(veh1.head_angle * np.pi / 180) + veh1.vy_initial * 1.46667 * np.cos(veh1.head_angle * np.pi / 180)
    print(f'Initial Kingpin velocity -> x: {kp_Vx * 0.681818:0.2f}, y: {kp_Vy * 0.681818:0.2f}')
    
    # kingpin velocity in trailer frame
    kp_vx = kp_Vx * np.cos(veh2.head_angle * np.pi / 180) - kp_Vy * np.sin(veh2.head_angle * np.pi / 180)
    kp_vy = -1 * kp_Vx * np.sin(veh2.head_angle * np.pi / 180) + kp_Vy * np.cos(veh2.head_angle * np.pi / 180)
    print(f'Initial Kingpin velocity in trailer frame -> x: {kp_vx * 0.681818:0.2f}, y: {kp_vy * 0.681818:0.2f}')
    
    # trailer omega
    veh2.omega_z = kp_vy / veh2.kp_axle
    print(f'Initial trailer omega: {veh2.omega_z :0.2f}')
    
    # trailer cg velocity
    veh2.vx_initial = kp_vx * 0.681818
    veh2.vy_initial = (kp_vy - veh2.lcgf * veh2.omega_z) * 0.681818
    print(f'Initial Trailer velocity -> x: {veh2.vx_initial:0.2f}, y: {veh2.vy_initial:0.2f} mph')
    
    # Vehicle 2 initial position based on impact point
    veh2_dx = 5
    veh2_dy = -1 * veh2.width / 2
    
    v2_hangle_rad = veh2.head_angle * np.pi / 180
    veh2.init_x_pos = -1 * (veh2_dx * np.cos(v2_hangle_rad) - veh2_dy * np.sin(v2_hangle_rad))
    veh2.init_y_pos = -1 * (veh2_dx * np.sin(v2_hangle_rad) + veh2_dy * np.cos(v2_hangle_rad))
    
    # impact point in vehicle 2 reference frame
    veh2.pimpact_x = veh2_dx
    veh2.pimpact_y = veh2_dy
    
    # Vehicle 1 based on location of trailer
    kp_Dx = veh2.init_x_pos + veh2.lcgf * np.cos(veh2.head_angle * np.pi / 180)
    kp_Dy = veh2.init_y_pos + veh2.lcgf * np.sin(veh2.head_angle * np.pi / 180)
    
    veh1.init_x_pos = kp_Dx + veh1.cg_kp * np.cos(veh1.head_angle * np.pi / 180)
    veh1.init_y_pos = kp_Dy + veh1.cg_kp * np.sin(veh1.head_angle * np.pi / 180)
    
    # Vehicle 3 initial position based on impact point
    veh3_dx = -1 * veh3.lcgr - veh3.r_hang
    veh3_dy = -1 * veh3.width / 2
    
    v3_hangle_rad = veh3.head_angle * np.pi / 180
    veh3.init_x_pos = -1 * (veh3_dx * np.cos(v3_hangle_rad) - veh3_dy * np.sin(v3_hangle_rad))
    veh3.init_y_pos = -1 * (veh3_dx * np.sin(v3_hangle_rad) + veh3_dy * np.cos(v3_hangle_rad))
    
    # impact point in vehicle 3 reference frame
    veh3.pimpact_x = veh3_dx
    veh3.pimpact_y = veh3_dy
    
    # inputs for simulation
    sim_inputs = {'cor': 0.2,
                 'cof': 0.3,
                 'impact_norm_deg': impact_norm_angle}
    
    name = 'Bus vs Trailer'
    run = IMPC(name, veh2, veh3, sim_inputs)

    base_output = {
                   'sim': n_run,
                   'v1_speed': veh1.vx_initial,
                   'v2x_speed': veh2.vx_initial,
                   'v2y_speed': veh2.vy_initial,
                   'v1_head_angle': veh1.head_angle,
                   'v2_head_angle': veh2.head_angle,
                   'impact_plane_angle': impact_norm_angle,
                   'v2_dvx': run.__dict__['v1_result']['dvx'],
                   'v2_dvy': run.__dict__['v1_result']['dvy'],
                   'v2_dv': run.__dict__['v1_result']['dv'],
                   'v3_dvx': run.__dict__['v2_result']['dvx'],
                   'v3_dvy': run.__dict__['v2_result']['dvy'],
                   'v3_dv': run.__dict__['v2_result']['dv']
                   }

    monte_carlo_impacts = pd.concat([
                                    monte_carlo_impacts,
                                    pd.DataFrame([
                                                  base_output                
                                                ])
                                    ],
                                                ignore_index=True
                                    )
    del base_output
    
    clear_output(wait=False)

monte_carlo_impacts.to_csv('/Users/joe/mckinnis_simulations.csv', index=False)

In [137]:
run.__dict__['v2_result']['vx']

1.11516050328014

In [138]:
run.__dict__

{'name': 'Bus vs Trailer',
 'type': 'impc_carpenter',
 'veh1': <pycrash.vehicle.Vehicle at 0x176f02f90>,
 'veh2': <pycrash.vehicle.Vehicle at 0x176f31350>,
 'poi_veh2x': -17.291666666666664,
 'poi_veh2y': -4.25,
 'v1_result': {'vx': 1.4455317545330766,
  'vy': -3.7475580405494306,
  'oz_rad': 0.014953916710315589,
  'oz_deg': 0.8567963146912394,
  'dvx': 0.13819036812344693,
  'dvy': 2.3961069406649806,
  'dv': 2.4000885502299676},
 'v2_result': {'vx': 1.11516050328014,
  'vy': 2.5892797038899436,
  'oz_rad': -0.1862860911129808,
  'oz_deg': -10.673406802763312,
  'dvx': 1.11516050328014,
  'dvy': 2.5892797038899436,
  'dv': 2.819211296276391}}